Lets define some example Django models to work with:

In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'tests.settings')

# Setup Django
django.setup()

from django.contrib.auth.models import User
from django.db import models


class Author(models.Model):
    name = models.CharField(max_length=100)
    birth_date = models.DateField()

    class Meta:
        app_label = "tests"

class Publisher(models.Model):
    name = models.CharField(max_length=100)
    address = models.TextField(help_text="Publisher's official address")

    class Meta:
        app_label = "tests"


class Book(models.Model):
    title = models.CharField(max_length=200)
    isbn = models.CharField(max_length=13, unique=True)
    publication_date = models.DateField()
    authors = models.ManyToManyField(Author)  # Many-to-Many relationship
    publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)  # Foreign Key relationship

    class Meta:
        app_label = "tests"
        default_related_name = 'books'


class Library(models.Model):
    name = models.CharField(max_length=100)
    address = models.TextField()

    class Meta:
        app_label = "tests"


class BookCopy(models.Model):
    book = models.ForeignKey(Book, on_delete=models.CASCADE)  # Foreign Key relationship
    library = models.ForeignKey(Library, on_delete=models.CASCADE)  # Foreign Key relationship
    inventory_number = models.CharField(max_length=20, unique=True)

    class Meta:
        app_label = "tests"
        default_related_name = "book_copies"


class Borrowing(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)  # Foreign Key relationship
    book_copy = models.ForeignKey(BookCopy, on_delete=models.CASCADE)  # Foreign Key relationship
    borrow_date = models.DateField()
    return_date = models.DateField(null=True, blank=True)

    class Meta:
        app_label = "tests"
        default_related_name = 'borrowings'



Now lets do some testing:

In [3]:
from typing import ClassVar

from superschema.schema import SuperSchema
from superschema.types import Infer, ModelFields
from rich import print_json

class BookSchema(SuperSchema):
    """Book schema example with nested fields."""

    class Meta(SuperSchema.Meta):
        """Here we define the model and the fields we want to infer."""

        model = Book
        fields: ClassVar[ModelFields] = {
            "title": Infer,
            "isbn": Infer,
            "publication_date": Infer,
            "authors": {"name": Infer},
            "publisher": {"name": Infer},
            "book_copies": {"library": Infer}, # note: here we use a reverse relation
        }

print_json(data=BookSchema.model_json_schema())



{
  "$defs": {
    "None_authors": {
      "properties": {
        "name": {
          "description": "Name",
          "maxLength": 100,
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "None_authors",
      "type": "object"
    },
    "None_book_copies": {
      "properties": {
        "library": {
          "description": "Library",
          "title": "Library",
          "type": "integer"
        }
      },
      "required": [
        "library"
      ],
      "title": "None_book_copies",
      "type": "object"
    },
    "None_publisher": {
      "properties": {
        "name": {
          "description": "Name",
          "maxLength": 100,
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "None_publisher",
      "type": "object"
    }
  },
  "properties": {
    "title": {
      "description": "Title",
      "maxLength": 200,
      "title": "Title",
      "type": "string"
    },
    "isbn": {
      "description": "Isbn",
      "maxLength": 13,
      "title": "Isbn",
      "type": "string"
    },
    "publication_date": {
      "description": "Publication Date",
      "format": "date",
      "title": "Publication Date",
      "type": "string"
    },
    "authors": {
      "anyOf": [
        {
          "items": {
            "$ref": "#/$defs/None_authors"
          },
          "type": "array"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "authors",
      "title": "authors"
    },
    "publisher": {
      "$ref": "#/$defs/None_publisher",
      "description": "publisher",
      "title": "publisher"
    },
    "book_copies": {
      "anyOf": [
        {
          "items": {
            "$ref": "#/$defs/None_book_copies"
          },
          "type": "array"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "book_copies",
      "title": "book_copies"
    }
  },
  "required": [
    "title",
    "isbn",
    "publication_date",
    "publisher"
  ],
  "title": "BookSchema",
  "type": "object"
}